In [ ]:
%%capture
#install dependencies and libraries
! pip install TTS
!sudo apt-get install espeak-ng
!pip install fastapi uvicorn pydantic pyngrok nest_asyncio
!pip install python-multipart
import subprocess
from fastapi import FastAPI,Response
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from fastapi.responses import StreamingResponse,FileResponse
from pydantic import BaseModel
from IPython.display import Audio
from google.colab import userdata
import uvicorn
import shutil
import nest_asyncio
from pyngrok import ngrok
import base64
import tempfile

nest_asyncio.apply()

In [ ]:
# ! tts --list_models #run to see available models
model="tts_models/en/ljspeech/glow-tts" #replace model with the model of your choice

In [ ]:
def tts_call(text: str, model: str=model):
  # Create a temporary file to save the output audio
  with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_file:
    out_path = temp_file.name

  # Run the TTS command
  subprocess.run(["tts", "--model_name", model, "--text", text, "--out_path", out_path])

  # Return the path to the saved audio file
  return out_path

In [ ]:
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

In [ ]:
class TextRequest(BaseModel):
    text: str

class OutputText(BaseModel):
  answer: str

In [ ]:
@app.get("/synthesize_audios")
def synthesize_audio(payload: TextRequest):
    # Call your TTS function to generate the audio file
    print("text:", payload.text)
    audio_file_path = tts_call(payload.text)

    # Return the audio file as the response
    return FileResponse(audio_file_path, media_type="audio/wav")

@app.get("/", response_model=OutputText)
async def root():
    return {"answer": "Hello from TTS"}

In [ ]:
# read NGROK_TOKEN from secrets section
NGROK_TOKEN = userdata.get('NGROK_TOKEN')
!ngrok config add-authtoken NGROK_TOKEN

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print("Public URL:", ngrok_tunnel.public_url)
uvicorn.run(app, port=8000)